# In this notebook I will Try to connect the rag model for pdf ingestion and use pinecone as vector database


In [1]:
# We will be using these PDF loaders but you can check out other loaded documents
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from rich import print
import re
from transformers import AutoTokenizer
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

load_dotenv()

EMBEDDING_MODEL = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# This is the name of the report which should be in the directory
# You can download the precise PDF I am using from here https://www.pc.gov.pk/uploads/archives/PSDP_2023-24.pdf

name = '../data/raw/Easy_recipes.pdf'

# This loader uses PyMuPDF
loader_py = PyMuPDFLoader(name)

# Storing the loaded documents as langChain Document object
pages_py = loader_py.load()


c:\Users\naran\anaconda3\envs\legal_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(pages_py[0].page_content)
print(len(pages_py))

Quick     and     easy    recipes         
          
Start     cooking         today!          
You       can     do      it!     
          
Cooking   meals   is      much    cheaper         than    eating          outside.        This    is      true    
everywhere      
but       even    more    in      Geneva...       Take    turns   and     cook    for     your    roommate        
and     your    
friends!          You’ll          save    even    more    money   –       and     the     kitchen         will    
probably        stay    
cleaner...        
          
I         have    selected        only    a       few     recipes         from    these   websites:       
http://www.studentrecipes.com/    
http://www.squidoo.com/studentfood#module55467342         
          
          
Bon       appétit!

15

In [3]:
# text splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    # shows how to seperate
    separator="\n",
    # Shows the document token length
    chunk_size=1000,
    # How much overlap should exist between documents
    chunk_overlap=150,
    # How to measure length
    length_function=len
)

# Applying the splitter
docs = text_splitter.split_documents(pages_py)

print(docs[0])
print(len(docs))

Document(
    metadata={
        'producer': 'Mac OS X 10.6.4 Quartz PDFContext',
        'creator': 'Microsoft Word',
        'creationdate': "D:20100809161642Z00'00'",
        'source': '../data/raw/Easy_recipes.pdf',
        'file_path': '../data/raw/Easy_recipes.pdf',
        'total_pages': 15,
        'format': 'PDF 1.3',
        'title': 'Easy recipes',
        'author': 'sylvie meynier',
        'subject': '',
        'keywords': '',
        'moddate': "D:20100809161642Z00'00'",
        'trapped': '',
        'page': 0
    },
    page_content='Quick\t\r \xa0and\t\r \xa0easy\t\r \xa0recipes\t\r \xa0\n\t\r \xa0\nStart\t\r \xa0cooking\t\r 
\xa0today!\t\r \xa0\nYou\t\r \xa0can\t\r \xa0do\t\r \xa0it!\t\r \xa0\n\t\r \xa0\nCooking\t\r \xa0meals\t\r 
\xa0is\t\r \xa0much\t\r \xa0cheaper\t\r \xa0than\t\r \xa0eating\t\r \xa0outside.\t\r \xa0This\t\r \xa0is\t\r 
\xa0true\t\r \xa0everywhere\t\r \xa0\nbut\t\r \xa0even\t\r \xa0more\t\r \xa0in\t\r \xa0Geneva...\t\r \xa0Take\t\r 
\xa0turns\t\r \xa0and\t\r \xa0cook\t\r \xa0for\t\r \xa0your\t\r \xa0roommate\t\r \xa0and\t\r \xa0your\t\r 
\xa0\nfriends!\t\r \xa0You’ll\t\r \xa0save\t\r \xa0even\t\r \xa0more\t\r \xa0money\t\r \xa0–\t\r \xa0and\t\r 
\xa0the\t\r \xa0kitchen\t\r \xa0will\t\r \xa0probably\t\r \xa0stay\t\r \xa0\ncleaner...\t\r \xa0\n\t\r \xa0\nI\t\r 
\xa0have\t\r \xa0selected\t\r \xa0only\t\r \xa0a\t\r \xa0few\t\r \xa0recipes\t\r \xa0from\t\r \xa0these\t\r 
\xa0websites:\t\r \xa0\nhttp://www.studentrecipes.com/\t\r 
\xa0\nhttp://www.squidoo.com/studentfood#module55467342\t\r \xa0\n\t\r \xa0\n\t\r \xa0\nBon\t\r \xa0appétit!'
)

29

In [4]:
def clean_text(text):
    # Remove non-printable characters
    text = re.sub(r'[^\x20-\x7E]', '', text)
    
    # Remove excessive spaces, newlines, and tabs
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove unwanted special characters but keep medical symbols
    text = re.sub(r'[^\w\s.,:;/%°±()\-]', '', text)  
    text = re.sub(r'[\t\r\xa0]', ' ', text)  # Replace \t, \r, and \xa0 with a space
    text = re.sub(r'\n+', '\n', text)  # Keep meaningful line breaks
    text = re.sub(r' +', ' ', text).strip()  # Remove excessive space

    return text  

In [6]:
from langchain_core.documents import Document
docs = [Document(clean_text(doc.page_content),metadata=doc.metadata) for doc in docs]

In [7]:
print(docs[0])

Document(
    metadata={
        'producer': 'Mac OS X 10.6.4 Quartz PDFContext',
        'creator': 'Microsoft Word',
        'creationdate': "D:20100809161642Z00'00'",
        'source': '../data/raw/Easy_recipes.pdf',
        'file_path': '../data/raw/Easy_recipes.pdf',
        'total_pages': 15,
        'format': 'PDF 1.3',
        'title': 'Easy recipes',
        'author': 'sylvie meynier',
        'subject': '',
        'keywords': '',
        'moddate': "D:20100809161642Z00'00'",
        'trapped': '',
        'page': 0
    },
    page_content='Quick and easy recipes Start cooking today You can do it Cooking meals is much cheaper than 
eating outside. This is true everywhere but even more in Geneva... Take turns and cook for your roommate and your 
friends Youll save even more money and the kitchen will probably stay cleaner... I have selected only a few recipes
from these websites: http://www.studentrecipes.com/ http://www.squidoo.com/studentfoodmodule55467342 Bon apptit'
)

In [8]:
embeddings=AutoTokenizer.from_pretrained(EMBEDDING_MODEL)

In [9]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
print(pinecone_api_key)
pc = Pinecone(api_key=pinecone_api_key)

pcsk_7PvMU4_15gJfjME25FYrtL2aWQR9k63AkdNHWynw3ZLXnPJ1kFVggdkLMRpESv5ajdTNfF

In [10]:
import time

index_name = "test-index-legal"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
print(existing_indexes)

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
print(index)

['test-index-legal']

<pinecone.data.index.Index object at 0x000001B43A5435B0>

In [11]:
index.upsert(items=docs)


TypeError: Index.upsert() missing 1 required positional argument: 'vectors'